In [1]:
# General libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 

## Preliminary work on the dataset

In [2]:
yahoo_df = pd.read_csv("../data/IBM.csv")
yahoo_df = yahoo_df.set_index("Date")

X = yahoo_df[['ROC_4', 'ROC_5', 'ROC_6', '4 Day ROI', '6 Day ROI', '30 Day ROI',
       'RSI_3', 'RSI_4', 'CCI']]
y = yahoo_df["Up down"]

# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)


# Normalize

In [3]:
def shift_norm(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [4]:
X_train_80 = shift_norm(X_train_80)
X_test = shift_norm(X_test)

In [5]:
X_train = shift_norm(X_train)
X_valid = shift_norm(X_valid)

# kNN

In [6]:
# Train and fit
kNN = KNeighborsClassifier()
kNN.fit(X_train, y_train)

# Predict
y_pred_train = kNN.predict(X_train)
y_pred_valid = kNN.predict(X_valid)

# Compute Accuracy
train_acc = accuracy_score(y_train,y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)

print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

Train Accuracy: 0.69130 - Validation Accuracy: 0.52105


### Tuning parameters

In [7]:
knn = KNeighborsClassifier()

k_range = list(range(1,31))
weight_options = ["uniform", "distance"]
param_grid = dict(n_neighbors = k_range, weights = weight_options)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
  
# fitting the model for grid search
grid_search=grid.fit(X_train, y_train)

In [8]:
print(grid_search.best_estimator_)

KNeighborsClassifier(n_neighbors=25)


In [9]:
# Train and fit
kNN = KNeighborsClassifier(n_neighbors=2)
kNN.fit(X_train_80, y_train_80)

# Predict
y_pred_test = kNN.predict(X_test)
y_prob = kNN.predict_proba(X_test)[:,1]

# Finally evaluate on test
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

# Store the Test Accuracy 
kNN_accuracy_test = test_acc

Test Accuracy: 0.50841


# Save data prediction

In [16]:
df_prediction = pd.read_csv("../data/model_accuracy_IBM.csv", index_col = False)
df_prediction["y"] = y_test.values
df_prediction["kNN_pred"] = y_pred_test
df_prediction["kNN_accuracy"] = [kNN_accuracy_test for x in range(len(y_pred_test))]
df_prediction["kNN_prob"] = y_prob

In [17]:
df_prediction = df_prediction.drop(columns=["Unnamed: 0"])
df_prediction

,LogReg_pred,LogReg_accuracy,LogReg_prob,LGBM_pred,LGBM_accuracy,LGBM_prob,y,kNN_pred,kNN_accuracy,kNN_prob,...,SVC_pred,SVC_accuracy,SVC_prob,LSTM_price_accuracy,RNN_pred,RNN_accuracy,RNN_prob,XGB_pred,XGB_accuracy,XGB_prob
0,0,0.48785,0.499232,0,0.48972,0.482038,0,0,0.508411,0.5,...,1,0.492523,0.517458,0.857009,0,0.493458,0.488459,1,0.496262,0.515965
1,1,0.48785,0.504807,1,0.48972,0.516175,1,0,0.508411,0.5,...,1,0.492523,0.517960,0.857009,1,0.493458,0.536730,1,0.496262,0.524946
2,1,0.48785,0.591117,1,0.48972,0.526401,1,0,0.508411,0.5,...,1,0.492523,0.526435,0.857009,1,0.493458,0.626067,1,0.496262,0.520137
3,1,0.48785,0.595272,1,0.48972,0.515859,1,1,0.508411,1.0,...,1,0.492523,0.527133,0.857009,1,0.493458,0.636652,0,0.496262,0.486497
4,1,0.48785,0.590655,1,0.48972,0.515859,1,1,0.508411,1.0,...,1,0.492523,0.526471,0.857009,1,0.493458,0.635811,1,0.496262,0.545093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1,0.48785,0.543262,1,0.48972,0.536454,1,0,0.508411,0.5,...,1,0.492523,0.522732,0.857009,1,0.493458,0.548477,0,0.496262,0.430383
1066,1,0.48785,0.570132,1,0.48972,0.521491,1,1,0.508411,1.0,...,1,0.492523,0.524813,0.857009,1,0.493458,0.591784,1,0.496262,0.592070
1067,1,0.48785,0.542180,1,0.48972,0.537546,0,1,0.508411,1.0,...,1,0.492523,0.522923,0.857009,1,0.493458,0.559243,1,0.496262,0.574436
1068,1,0.48785,0.569741,1,0.48972,0.521313,1,0,0.508411,0.5,...,1,0.492523,0.524824,0.857009,1,0.493458,0.595539,1,0.496262,0.584061


In [18]:
df_prediction.to_csv("../data/model_accuracy_IBM.csv")